### 데이터 가져오기

In [1]:
import pandas as pd

df = pd.read_csv("../../db/news_2023_2025_summarized.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    14087 non-null  int64 
 1   news_id                       14087 non-null  object
 2   wdate                         14087 non-null  object
 3   title                         14087 non-null  object
 4   article                       14087 non-null  object
 5   press                         14087 non-null  object
 6   url                           14087 non-null  object
 7   image                         14087 non-null  object
 8   article_preprocessed          14087 non-null  object
 9   summary                       14087 non-null  object
 10  stock_list                    14087 non-null  object
 11  industry_list                 14087 non-null  object
 12  summary_embedding             14087 non-null  object
 13  summary_embeddin

In [2]:
rate_df = pd.read_csv("../../db/korea_base_rate_daily.csv")

## 함수 모듈화

In [91]:
import pandas as pd
import numpy as np
import ast
import os
from pykrx import stock
import requests
from datetime import timedelta

class NewsMarketPipeline:
    def __init__(self, df, api_key, rate_path=None):
        print("[초기화] 파이프라인 객체 생성")
        self.api_key = api_key
        self.rate_path = rate_path

        self.df = df
        self.ticker_name_map = None
        self.trading_days = None
        self.ohlcv_dict = {}
        self.trading_dict = {}
        self.fx_df = None
        self.bond_df = None
        self.rate_df = None

    def get_df(self):
        return self.df

    def extract_stock_name(self):

        print("[전처리] 종목명 추출")
        def extract_last(labels):
            if pd.isna(labels) or labels == '[]' or labels == '':
                return None
            if isinstance(labels, str):
                try:
                    labels_list = ast.literal_eval(labels)
                except Exception:
                    return None
            elif isinstance(labels, list):
                labels_list = labels
            else:
                return None
            if not labels_list:
                return None
            return labels_list[-1]
        if 'stock_list' not in self.df.columns:
            raise Exception("stock_list 컬럼이 없습니다. 실제 컬럼: " + str(self.df.columns.tolist()))

        self.df['stock_name'] = self.df['stock_list'].apply(extract_last)
        print("[전처리] stock_name 컬럼 생성 완료")

    def add_news_date(self):
        print("[전처리] news_date 컬럼 생성")
        if 'wdate' in self.df.columns:
            self.df['wdate'] = pd.to_datetime(self.df['wdate'])
            self.df['news_date'] = self.df['wdate'].dt.normalize()
        elif 'news_date' in self.df.columns:
            self.df['news_date'] = pd.to_datetime(self.df['news_date'])
        else:
            raise Exception("wdate/news_date 컬럼이 없습니다. 실제 컬럼: " + str(self.df.columns.tolist()))
        print("[전처리] news_date 컬럼 생성 완료")

    def get_ticker_name_map(self, recent_date="2025-05-30"):
        print("[파생] KOSPI 티커-이름 맵핑 생성")
        kospi_tickers = stock.get_market_ticker_list(date=recent_date, market="KOSPI")
        mapping = {stock.get_market_ticker_name(ticker): ticker for ticker in kospi_tickers}
        print(f"[파생] 티커-이름 맵핑 완료: {len(mapping)}개")
        return mapping

    def add_ticker(self):
        print("[파생] ticker 컬럼 생성")
        if self.ticker_name_map is None:
            self.ticker_name_map = self.get_ticker_name_map()
        def get_ticker(name):
            if pd.isna(name):
                return None
            return self.ticker_name_map.get(name)
        self.df['ticker'] = self.df['stock_name'].apply(get_ticker)
        print("[파생] ticker 컬럼 생성 완료")

    def get_trading_days(self, start_year=2022, end_year=2026):
        print("[파생] 거래일 리스트 생성")
        days = []
        for y in range(start_year, end_year + 1):
            for m in range(1, 13):
                try:
                    days_this_month = stock.get_business_days(y, m)
                    days.extend(days_this_month)
                except Exception as e:
                    print(f"{y}-{m:02d}: 거래일 없음 또는 에러 ({e})")
        print("[파생] 거래일 리스트 생성 완료")
        return pd.to_datetime(sorted(set(days)))

    def find_nearest_trading_day(self, date):
        after = self.trading_days[self.trading_days >= date]
        return after[0] if len(after) > 0 else pd.NaT

    def add_trading_dates(self):
        print("[파생] 거래일 기준 컬럼 생성")
        if self.trading_days is None:
            self.trading_days = self.get_trading_days()
        self.df['D_day_date'] = self.df['news_date'].apply(self.find_nearest_trading_day)

        offsets = {
            'D_minus_14_date': -14, 'D_minus_7_date': -7, 'D_minus_3_date': -3,
            'D_minus_2_date': -2, 'D_minus_1_date': -1, 'D_day_date': 0,
            }
        def fill_offsets(row):
            d_day = row['D_day_date']
            res = {}
            if pd.isna(d_day) or d_day not in self.trading_days.values:
                for k in offsets.keys():
                    res[k] = pd.NaT
                return pd.Series(res)
            idx = np.where(self.trading_days == d_day)[0][0]
            for k, v in offsets.items():
                i = idx + v
                res[k] = self.trading_days[i] if 0 <= i < len(self.trading_days) else pd.NaT
            return pd.Series(res)
        df_offsets = self.df.apply(fill_offsets, axis=1)

        self.df.drop(columns=['D_day_date'], inplace=True)

        # 인덱스 리셋 후 concat (중복 방지)
        self.df = self.df.reset_index(drop=True)
        df_offsets = df_offsets.reset_index(drop=True)
        self.df = pd.concat([self.df, df_offsets], axis=1)
        print("[파생] 거래일 기준 컬럼 생성 완료")

    def fetch_ohlcv_and_trading(self):
        print("[수집] OHLCV/투자자별 순매수 데이터 수집")
        offsets = [
            'D_minus_14_date','D_minus_7_date','D_minus_3_date','D_minus_2_date','D_minus_1_date',
        ]
        all_dates = pd.concat([self.df[col] for col in offsets], ignore_index=True).dropna().unique()
        all_dates_str = sorted([d.strftime('%Y%m%d') for d in pd.to_datetime(all_dates)])
        tickers = self.df['ticker'].dropna().unique().tolist()
        for ticker in tickers:
            try:
                ohlcv = stock.get_market_ohlcv_by_date(min(all_dates_str), max(all_dates_str), ticker)
                self.ohlcv_dict[ticker] = ohlcv
            except Exception as e:
                print(f"[WARN] OHLCV 조회 실패: {ticker} ({e})")
            try:
                tv = stock.get_market_trading_value_by_date(min(all_dates_str), max(all_dates_str), ticker)
                self.trading_dict[ticker] = tv
            except Exception as e:
                print(f"[WARN] Trading Value 조회 실패: {ticker} ({e})")
        print("[수집] OHLCV/투자자별 순매수 데이터 수집 완료")

    def add_ohlcv_and_trading(self):
        print("[파생] OHLCV/투자자별 순매수 컬럼 생성")
        offsets = [
            'D_minus_14_date','D_minus_7_date','D_minus_3_date','D_minus_2_date','D_minus_1_date',
        ]
        def get_ohlcv_val(row, date_col, val_col):
            ticker = row['ticker']
            date = row[date_col]
            if pd.isna(ticker) or pd.isna(date):
                return np.nan
            df_ohlcv = self.ohlcv_dict.get(ticker)
            if df_ohlcv is None:
                return np.nan
            date_str = date.strftime('%Y%m%d')
            if date_str not in df_ohlcv.index:
                return np.nan
            return df_ohlcv.loc[date_str, val_col]
        def get_trading_val(row, date_col, investor):
            ticker = row['ticker']
            date = row[date_col]
            if pd.isna(ticker) or pd.isna(date):
                return np.nan
            df_tv = self.trading_dict.get(ticker)
            if df_tv is None:
                return np.nan
            date_str = date.strftime('%Y%m%d')
            if date_str not in df_tv.index:
                return np.nan
            col_map = {'외국인': '외국인합계', '기관': '기관합계', '개인': '개인'}
            return df_tv.loc[date_str, col_map[investor]]
        for col in offsets:
            self.df[f'{col}_close'] = self.df.apply(lambda r: get_ohlcv_val(r, col, '종가'), axis=1)
            self.df[f'{col}_volume'] = self.df.apply(lambda r: get_ohlcv_val(r, col, '거래량'), axis=1)
            self.df[f'{col}_foreign'] = self.df.apply(lambda r: get_trading_val(r, col, '외국인'), axis=1)
            self.df[f'{col}_institution'] = self.df.apply(lambda r: get_trading_val(r, col, '기관'), axis=1)
            self.df[f'{col}_individual'] = self.df.apply(lambda r: get_trading_val(r, col, '개인'), axis=1)
        print("[파생] OHLCV/투자자별 순매수 컬럼 생성 완료")

    def fetch_fx(self, start_date, end_date):
        print("[수집] 환율 데이터")
        if self.fx_df is not None:
            return self.fx_df
        stat_code = "731Y001"
        item_code = "0000001"
        print(start_date, end_date)
        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{self.api_key}/json/kr/1/1000/{stat_code}/D/{start_date}/{end_date}/{item_code}/"
        resp = requests.get(url)
        data = resp.json()
        if "StatisticSearch" not in data or "row" not in data["StatisticSearch"]:
            print("[WARN] 환율 데이터 없음:", data)
            return pd.DataFrame()
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df['date'] = pd.to_datetime(df['TIME'], format='%Y%m%d')
        df['usdkrw'] = pd.to_numeric(df['DATA_VALUE'], errors='coerce')
        df = df.sort_values('date')
        self.fx_df = df[['date', 'usdkrw']]
        print("[수집] 환율 데이터 수집 완료")
        return self.fx_df

    def fetch_bond10y(self, start_date, end_date):
        print("[수집] 국채10년 데이터")
        if self.bond_df is not None:
            return self.bond_df
        stat_code = "817Y002"
        item_code = "010200000"
        url = f"https://ecos.bok.or.kr/api/StatisticSearch/{self.api_key}/json/kr/1/1000/{stat_code}/D/{start_date}/{end_date}/{item_code}/"
        resp = requests.get(url)
        data = resp.json()
        if "StatisticSearch" not in data or "row" not in data["StatisticSearch"]:
            print("[WARN] 국채10년 데이터 없음:", data)
            return pd.DataFrame()
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df['date'] = pd.to_datetime(df['TIME'], format='%Y%m%d')
        df['bond10y'] = pd.to_numeric(df['DATA_VALUE'], errors='coerce')
        df = df.sort_values('date')
        self.bond_df = df[['date', 'bond10y']]
        print("[수집] 국채10년 데이터 수집 완료")
        return self.bond_df

    def load_rate_df(self):
        if self.rate_path:
            print("[수집] 기준금리 데이터")
            self.rate_df = pd.read_csv(self.rate_path)
            self.rate_df['date'] = pd.to_datetime(self.rate_df['date'])
            self.rate_df = self.rate_df.sort_values('date')
            print("[수집] 기준금리 데이터 수집 완료")

    def add_external_vars(self):
        print("[병합] 외부 변수(환율, 국채, 기준금리) 병합")
        self.df = self.df.sort_values('news_date')
        start_date = (self.df["news_date"].min() - timedelta(days=1)).strftime("%Y%m%d")
        end_date = (self.df["news_date"].max() - timedelta(days=1)).strftime("%Y%m%d")
        fx_df = self.fetch_fx(start_date, end_date)
        bond_df = self.fetch_bond10y(start_date, end_date)
        self.load_rate_df()
        fx_df = fx_df.sort_values('date')
        bond_df = bond_df.sort_values('date')
        self.df = pd.merge_asof(self.df, fx_df.rename(columns={'date': 'news_date', 'usdkrw': 'fx'}), on='news_date', direction='backward')
        self.df = pd.merge_asof(self.df, bond_df.rename(columns={'date': 'news_date'}), on='news_date', direction='backward')
        if self.rate_df is not None:
            self.df = pd.merge_asof(self.df, self.rate_df.rename(columns={'date': 'news_date', 'rate': 'base_rate'}), on='news_date', direction='backward')
        print("[병합] 외부 변수 병합 완료")

    def run(self):
        self.extract_stock_name()
        self.add_news_date()
        self.add_ticker()
        self.add_trading_dates()
        self.fetch_ohlcv_and_trading()
        self.add_ohlcv_and_trading()
        self.add_external_vars()
        print("[완료] 전체 파이프라인 완료")
        print(self.df.head())
        return self.df

In [103]:
news = {
    "wdate": "2025-06-11 22:52:00",
    'stock_list': '["삼성전자"]'
}

In [104]:
pd.DataFrame([news])

,wdate,stock_list
0,2025-06-11 22:52:00,"[""삼성전자""]"


In [105]:
import os
from dotenv import load_dotenv

load_dotenv()

# 사용 예시

pipeline = NewsMarketPipeline(

    df=pd.DataFrame([news]),

    api_key=os.getenv('KOREA_BANK_API_KEY'),

    rate_path="../../db/korea_base_rate_daily.csv",
)

final_df = pipeline.run()

final_df.head()

[초기화] 파이프라인 객체 생성
[전처리] 종목명 추출
[전처리] stock_name 컬럼 생성 완료
[전처리] news_date 컬럼 생성
[전처리] news_date 컬럼 생성 완료
[파생] ticker 컬럼 생성
[파생] KOSPI 티커-이름 맵핑 생성
[파생] 티커-이름 맵핑 완료: 961개
[파생] ticker 컬럼 생성 완료
[파생] 거래일 기준 컬럼 생성
[파생] 거래일 리스트 생성


C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_busi

2025-07: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-08: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-09: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-10: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_busi

2026-01: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-02: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-03: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-04: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-05: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


...

C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_busi

2026-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
[파생] 거래일 리스트 생성 완료
[파생] 거래일 기준 컬럼 생성 완료
[수집] OHLCV/투자자별 순매수 데이터 수집
[수집] OHLCV/투자자별 순매수 데이터 수집 완료
[파생] OHLCV/투자자별 순매수 컬럼 생성
[파생] OHLCV/투자자별 순매수 컬럼 생성 완료
[병합] 외부 변수(환율, 국채, 기준금리) 병합
[수집] 환율 데이터
20250610 20250610


C:\Users\user\AppData\Local\Temp\ipykernel_2768\808006865.py:86: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)


[수집] 환율 데이터 수집 완료
[수집] 국채10년 데이터
[수집] 국채10년 데이터 수집 완료
[수집] 기준금리 데이터
[수집] 기준금리 데이터 수집 완료
[병합] 외부 변수 병합 완료
[완료] 전체 파이프라인 완료
                wdate stock_list stock_name  news_date  ticker  \
0 2025-06-11 22:52:00   ["삼성전자"]       삼성전자 2025-06-11  005930   

  D_minus_14_date D_minus_7_date D_minus_3_date D_minus_2_date D_minus_1_date  \
0      2025-05-20     2025-05-29     2025-06-05     2025-06-09     2025-06-10   

   ... D_minus_2_date_institution  D_minus_2_date_individual  \
0  ...              -129336955150              -159736050350   

   D_minus_1_date_close  D_minus_1_date_volume  D_minus_1_date_foreign  \
0               59200.0             15305760.0            -54533069200   

   D_minus_1_date_institution  D_minus_1_date_individual      fx  bond10y  \
0                -17931077300                71865041600  1358.9    2.385   

   base_rate  
0        2.5  

[1 rows x 39 columns]


,wdate,stock_list,stock_name,news_date,ticker,D_minus_14_date,D_minus_7_date,D_minus_3_date,D_minus_2_date,D_minus_1_date,...,D_minus_2_date_institution,D_minus_2_date_individual,D_minus_1_date_close,D_minus_1_date_volume,D_minus_1_date_foreign,D_minus_1_date_institution,D_minus_1_date_individual,fx,bond10y,base_rate
0,2025-06-11 22:52:00,"[""삼성전자""]",삼성전자,2025-06-11,005930,2025-05-20,2025-05-29,2025-06-05,2025-06-09,2025-06-10,...,-129336955150,-159736050350,59200.0,15305760.0,-54533069200,-17931077300,71865041600,1358.9,2.385,2.5


In [107]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   wdate                        1 non-null      datetime64[ns]
 1   stock_list                   1 non-null      object        
 2   stock_name                   1 non-null      object        
 3   news_date                    1 non-null      datetime64[ns]
 4   ticker                       1 non-null      object        
 5   D_minus_14_date              1 non-null      datetime64[ns]
 6   D_minus_7_date               1 non-null      datetime64[ns]
 7   D_minus_3_date               1 non-null      datetime64[ns]
 8   D_minus_2_date               1 non-null      datetime64[ns]
 9   D_minus_1_date               1 non-null      datetime64[ns]
 10  D_day_date                   1 non-null      datetime64[ns]
 11  D_minus_14_date_close        1 non-null      floa

In [65]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 87 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

In [66]:
final_df.tail(10)

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,...,D_plus_14_date_volume,D_plus_14_date_foreign,D_plus_14_date_institution,D_plus_14_date_individual,D_day_date_open,D-day_change_open,D-day_change,fx,bond10y,base_rate
14077,20250523_0076,2025-05-23 08:44:00,자발적 '상장폐지' 선언한 회사…알고 봤더니 '깜짝' [종목+],회삿돈으로 상장폐지?…텔코웨어 자진 상폐 논란\n주주가치 제고 명목으로 사들인 자사...,한국경제,https://n.news.naver.com/mnews/article/015/000...,https://imgnews.pstatic.net/image/015/2025/05/...,회삿돈으로 상장폐지?…텔코웨어 자진 상폐 논란 주주가치 제고 명목으로 사들인 자사주...,금융감독원 전자공시시스템에 따르면 텔코웨어의 금한태 대표는 지난 19일부터 자사주를...,['텔코웨어'],...,NaN,NaN,NaN,NaN,12920.0,0.077399,0.154919,1378.4,2.338,2.75
14078,20250523_0084,2025-05-23 08:33:00,"한화투자증권 ""제주항공, 2분기 적자 지속…목표가↓""","제주항공 항공기\n[제주항공 제공, 재판매 및 DB 금지]\n(서울=연합뉴스) 이민...",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2025/05/...,제주항공 항공기 (서울=연합뉴스) 이민영 기자 = 한화투자증권은 23일 제주항공에...,한화투자증권은 23일 제주항공에 대해 소비자 심리 회복까지 시간이 필요해 올해 2분...,"['한화투자증권', '제주항공']",...,NaN,NaN,NaN,NaN,6810.0,-1.468429,-1.756955,1378.4,2.338,2.75
14079,20250523_0086,2025-05-23 08:31:00,"하나증권, 한화솔루션 목표가↓ ""美 IRA 수정안 통과 우려""",한화솔루션\n[OCI 제공. DB 및 재판매 금지]\n(서울=연합뉴스) 곽윤아 기자...,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2025/05/...,미국 하원은 간밤 본회의에서 IRA 수정안을 통과시켰다. 하나증권에 따르면 이후 남...,하나증권에 따르면 미국 하원은 간밤 본회의에서 IRA 수정안을 통과시켰으며 이후 남...,['한화솔루션'],...,NaN,NaN,NaN,NaN,31450.0,-6.200318,-11.411411,1378.4,2.338,2.75
14080,20250523_0087,2025-05-23 08:30:00,"NH투자 ""제약·바이오 올 하반기 미국發 불확실성 클 것""",미국의 한 제약 바이오 연구실 [자료사진]\n(AFP=연합뉴스)\n(서울=연합뉴스)...,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2025/05/...,미국의 한 제약 바이오 연구실 (AFP=연합뉴스) (서울=연합뉴스) 김태균 기자 ...,"23일 NH투자증권은 올해 하반기 제약·바이오 산업의 전망과 관련해 ""미국 헬스케어...","['NH투자증권', '한미약품']",...,NaN,NaN,NaN,NaN,289500.0,0.863558,1.213172,1378.4,2.338,2.75
14081,20250523_0097,2025-05-23 08:00:00,"""SPC삼립, 반복되는 안전사고…투자의견·목표가↓""-IBK",사진=연합뉴스\nIBK투자증권은 23일 SPC삼립에 대해 반복되는 안전사고로 악화된...,한국경제,https://n.news.naver.com/mnews/article/015/000...,https://imgnews.pstatic.net/image/015/2025/05/...,지난 19일 오전 3시께 경기 시흥시에 있는 SPC삼립의 시화공장에서 50대 여성 ...,지난 19일 경기 시흥시에 있는 SPC삼립의 시화공장에서 환경·사회·지배구조 리스크...,['SPC삼립'],...,NaN,NaN,NaN,NaN,52500.0,-1.333333,-2.079395,1378.4,2.338,2.75
14082,20250523_0104,2025-05-23 07:51:00,"[클릭 e종목]""롯데지주, 예상보다 느린 실적 개선…목표가 유지""","IBK투자증권은 23일 롯데지주에 대해 ""1분기 예상보다 느린 실적 개선을 보였지만...",아시아경제,https://n.news.naver.com/mnews/article/277/000...,https://imgnews.pstatic.net/image/277/2025/05/...,"또한 ""바이오로직스 투자가 비용 부담으로 이어졌고, 롯데케미칼 실적 부진도 계속됐다...","롯데GRS, 종속회사 롯데쇼핑의 실적 개선이 고무적으로 이뤄졌지만, 내수 부문 실적...",['롯데쇼핑'],...,NaN,NaN,NaN,NaN,77300.0,0.258732,-0.641026,1378.4,2.338,2.75
14083,20250523_0109,2025-05-23 07:46:00,"""삼성바이오로직스 분할, 신약 개발 본격화를 의미""-키움",사진=한경DB\n키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본...,한국경제,https://n.news.naver.com/mnews/article/015/000...,https://imgnews.pstatic.net/image/015/2025/05/...,사진=한경DB 키움증권은 23일 삼성바이오로직스에 대해 인적분할을 추진하는 건 본격...,키움증권은 23일 삼성바이오로직스가 의약품 위탁 개발·생산(CDMO) 부문은 존속법...,"['키움증권', '삼성바이오로직스']",...,NaN,NaN,NaN,NaN,1080000.0,-5.925926,-5.925926,1378.4,2.338,2.75
14084,20250523_0116,2025-05-23 07:29:00,"[클릭 e종목]""크래프톤, 이제 서브노티카2에 빠질 시간""",IBK투자증권이 23일 크래프톤에 대해 최근 출시한 '인조이'가 양호한 성과를 기록...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,https://imgnews.pstatic.net/image/277/2025/05/...,"크래프톤은 '인조이', '서브노티카2', '딩컴', 'Project AB' 등 신작...",크래프톤은 신작 라인업을 앞세우며 기존 PUBG 중심에서 벗어나 게임 포트폴리오를 ...,['크래프톤'],...,NaN,NaN,NaN,NaN,378500.0,0.264201,-0.131579,1378.4,2.338,2.75
14085,20250523_0071,2025-05-23 08:52:00,"삼성생명, 그룹 지배구조 재편 가능성에 재부각…""지켜봐야""-LS",서울 서초구 삼성생명 본사. /사진=뉴스1 /사진=(서울=뉴스1) 김성진 기자\nL...,머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2025/05/...,서울 서초구 삼성생명 본사. = 매각대금은 원칙적으로 손실계약 보전에 사용될 수 없...,금리인하·제도강화 영향으로 매분기 자본 감소세가 이어지는 가운데 삼성전자 지분 보유...,['삼성전자'],...,NaN,NaN,NaN,NaN,55000.0,-1.454545,-0.914077,1378.4,2.338,2.75
14086,20250523_0002,2025

### 확인

In [67]:
news_df = pd.read_csv("/Users/JooAnLee/final_project/db/news(23-25)_summarized_external.csv")
final_df.equals(news_df)

False

In [70]:
for col in final_df.columns:
    if col not in news_df.columns:
        print(f"news_df에 '{col}' 컬럼이 없습니다.")
        continue
    # 컬럼 값 비교 (NaN도 같다고 처리하려면 fillna 처리 가능)
    diff_mask = final_df[col] != news_df[col]
    diff_mask = diff_mask.fillna(False)  # NaN vs NaN는 같다고 처리
    
    num_diff = diff_mask.sum()
    if num_diff == 0:
        print(f"컬럼 '{col}'는 모두 같습니다.")
    else:
        print(f"컬럼 '{col}'는 {num_diff}개 행에서 다릅니다.")
        # 다른 행 몇 개 예시 출력
        print(final_df.loc[diff_mask, col].head())
        print(news_df.loc[diff_mask, col].head())

컬럼 'news_id'는 13989개 행에서 다릅니다.
5    20240102_0253
6    20240102_0246
7    20240102_0237
8    20240102_0232
9    20240102_0231
Name: news_id, dtype: object
5    20240102_0333
6    20240102_0373
7    20240102_0413
8    20240102_0417
9    20240102_0411
Name: news_id, dtype: object
컬럼 'wdate'는 13985개 행에서 다릅니다.
5   2024-01-02 10:15:00
6   2024-01-02 10:16:00
7   2024-01-02 10:29:00
8   2024-01-02 10:30:00
9   2024-01-02 10:30:00
Name: wdate, dtype: datetime64[ns]
5    2024-01-02 09:03:00
6    2024-01-02 08:06:00
7    2024-01-02 07:00:00
8    2024-01-02 06:00:00
9    2024-01-02 07:35:00
Name: wdate, dtype: object
컬럼 'title'는 13989개 행에서 다릅니다.
5           궈밍쩡 유안타증권 대표 "제구포신 자세로 임하자"
6           코스피로 이동한 포스코DX…단기급등에 '숨고르기'
7              셀트리온 15% 급등…셀트리온제약 '상한가'
8    곽노정 SK하이닉스 사장 "내실 강화·과감한 투자 병행할 것"
9            NHN클라우드, 김동훈 단독 대표 체제 ‘출범’
Name: title, dtype: object
5                   포스코DX, 새해 첫 거래일 코스피서 거래
6    [클릭e종목]"태영건설 사태, 대규모 PF 위기 유발 가능성 제한적"
7           [Good딜Bad딜]新금융권력 사모펀드…MBK가 왜그

In [75]:
pd.set_option('display.max_columns', None)  # 모든 컬럼 출력 허용
pd.set_option('display.width', 2000)       # 가로 폭 충분히 넓게 설정
final_df.iloc[10000:10010]

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,industry_list,summary_embedding,stock_name,news_date,ticker,D_minus_14_date,D_minus_7_date,D_minus_3_date,D_minus_2_date,D_minus_1_date,D_day_date,D_plus_1_date,D_plus_2_date,D_plus_3_date,D_plus_7_date,D_plus_14_date,D_minus_14_date_close,D_minus_14_date_volume,D_minus_14_date_foreign,D_minus_14_date_institution,D_minus_14_date_individual,D_minus_7_date_close,D_minus_7_date_volume,D_minus_7_date_foreign,D_minus_7_date_institution,D_minus_7_date_individual,D_minus_3_date_close,D_minus_3_date_volume,D_minus_3_date_foreign,D_minus_3_date_institution,D_minus_3_date_individual,D_minus_2_date_close,D_minus_2_date_volume,D_minus_2_date_foreign,D_minus_2_date_institution,D_minus_2_date_individual,D_minus_1_date_close,D_minus_1_date_volume,D_minus_1_date_foreign,D_minus_1_date_institution,D_minus_1_date_individual,D_day_date_close,D_day_date_volume,D_day_date_foreign,D_day_date_institution,D_day_date_individual,D_plus_1_date_close,D_plus_1_date_volume,D_plus_1_date_foreign,D_plus_1_date_institution,D_plus_1_date_individual,D_plus_2_date_close,D_plus_2_date_volume,D_plus_2_date_foreign,D_plus_2_date_institution,D_plus_2_date_individual,D_plus_3_date_close,D_plus_3_date_volume,D_plus_3_date_foreign,D_plus_3_date_institution,D_plus_3_date_individual,D_plus_7_date_close,D_plus_7_date_volume,D_plus_7_date_foreign,D_plus_7_date_institution,D_plus_7_date_individual,D_plus_14_date_close,D_plus_14_date_volume,D_plus_14_date_foreign,D_plus_14_date_institution,D_plus_14_date_individual,D_day_date_open,D-day_change_open,D-day_change,fx,bond10y,base_rate
10000,20241217_0171,2024-12-17 09:53:00,"키움證 ""은행주, 정치적 리스크에도 주주환원 강화 지속 전망""",연합뉴스 제공\n[파이낸셜뉴스] 키움증권이 은행 업종에 대해 주주환원책이 지속될 전...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/12/...,연합뉴스 제공 키움증권이 은행 업종에 대해 주주환원책이 지속될 전망이라며 투자 의...,키움증권이 은행 업종에 대해 주주환원책이 지속될 전망이라며 투자 의견 비중 확대를 ...,['키움증권'],['금융 지원 서비스업'],"[-0.9721143841743469, -0.24344240128993988, 0....",키움증권,2024-12-17,039490,2024-11-27,2024-12-06,2024-12-12,2024-12-13,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-27,2025-01-09,127900.0,41409.0,1.639373e+09,1.964056e+09,-3.592969e+09,119200.0,102797.0,-9.711442e+08,-9.783141e+08,2.463373e+09,120500.0,143819.0,6.762204e+08,-1.565963e+09,8.058101e+08,122800.0,52353.0,-4.113789e+08,6.268026e+08,-7.019086e+08,120800.0,29587.0,-6.329323e+08,-2.675895e+08,9.037974e+08,120200.0,67359.0,-3.831999e+08,-5.773307e+08,9.067317e+08,126500.0,146729.0,6.619778e+08,3.393160e+09,-3.774690e+09,123000.0,116477.0,-4.784406e+09,-1.469154e+09,6.174531e+09,123700.0,60199.0,-1.480368e+09,1.673580e+09,-1.959089e+08,117300.0,198281.0,-6.617822e+09,-2.915770e+09,1.053060e+10,120000.0,69141.0,-3.504520e+08,7.758306e+08,-5.842680e+07,121500.0,-1.069959,-0.496689,1434.9,2.621,3.0
10001,20241217_0157,2024-12-17 10:20:00,"미래에셋증권, 'S&P DJSI World Index' 13년 연속 편입",미래에셋증권 센터원 빌딩 전경. 미래에셋증권 제공\n[파이낸셜뉴스] 미래에셋증권은 ...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/12/...,미래에셋증권 센터원 빌딩 전경. 증권업이 속한 다각화된 금융서비스부문(Diversi...,증권업이 속한 다각화된 금융서비스부문(Diversified Financial Ser...,['미래에셋증권'],['금융 지원 서비스업'],"[-0.9345676898956299, 0.45324188470840454, -0....",미래에셋증권,2024-12-17,006800,2024-11-27,2024-12-06,2024-12-12,2024-12-13,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-27,2025-01-09,8560.0,362455.0,1.549286e+08,1.244836e+09,-1.397030e+09,8420.0,652773.0,-8.327697e+08,7.635248e+08,7.269211e+07,8900.0,1102463.0,2.304853e+09,-9.908033e+08,-1.313786e+09,8670.0,494920.0,-9.694461e+08,-4.213359e+08,1.387184e+09,8590.0,377859.0,-8.281198e+08,-2.921555e+07,8.505325e+08,8400.0,509461.0,-1.562009e+09,-3.481589e+07,1.596825e+09,8620.0,490887.0,1.840672e+08,4.195753e+08,-6.047832e+08,8450.0,370381.0,6.146762e+08,-8.388230e+08,2.226705e+08,8050.0,1194406.0,-3.211277e+09,-9.910657e+08,5.078255e+09,8100.0,530621.0,5.685413e+07,-1.183038e

In [77]:
final_df.info()
print("==========================================================")
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 87 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   news_id                      14087 non-null  object        
 1   wdate                        14087 non-null  datetime64[ns]
 2   title                        14087 non-null  object        
 3   article                      14087 non-null  object        
 4   press                        14087 non-null  object        
 5   url                          14087 non-null  object        
 6   image                        14087 non-null  object        
 7   article_preprocessed         14087 non-null  object        
 8   summary                      14087 non-null  object        
 9   stock_list                   14087 non-null  object        
 10  industry_list                14087 non-null  object        
 11  summary_embedding            14087 non-nu

In [50]:
newsMarketPipeline = NewsMarketPipeline(
    news_path='/Users/JooAnLee/final_project/db/news_2023_2025_summarized.csv',
    api_key="BFKGNG1KJWZR9DVAIN6K",
    rate_path="/Users/JooAnLee/final_project/db/korea_base_rate_daily.csv"
)

[초기화] 파이프라인 객체 생성


In [51]:
newsMarketPipeline.load_news_df()

[로드] 뉴스 데이터: /Users/JooAnLee/final_project/db/news_2023_2025_summarized.csv
[로드] 컬럼: ['news_id', 'wdate', 'title', 'article', 'press', 'url', 'image', 'article_preprocessed', 'summary', 'stock_list', 'industry_list', 'summary_embedding']
[로드] 데이터 수: 14087


In [52]:
newsMarketPipeline.extract_stock_name()

[전처리] 종목명 추출
[전처리] stock_name 컬럼 생성 완료


In [53]:
newsMarketPipeline.add_news_date()

[전처리] news_date 컬럼 생성
[전처리] news_date 컬럼 생성 완료


In [54]:
newsMarketPipeline.get_ticker_name_map()

[파생] KOSPI 티커-이름 맵핑 생성
[파생] 티커-이름 맵핑 완료: 961개


{'AJ네트웍스': '095570',
 'AK홀딩스': '006840',
 'BGF': '027410',
 'BGF리테일': '282330',
 'BNK금융지주': '138930',
 'BYC': '001460',
 'BYC우': '001465',
 'CJ': '001040',
 'CJ CGV': '079160',
 'CJ4우(전환)': '00104K',
 'CJ대한통운': '000120',
 'CJ씨푸드': '011150',
 'CJ씨푸드1우': '011155',
 'CJ우': '001045',
 'CJ제일제당': '097950',
 'CJ제일제당 우': '097955',
 'CR홀딩스': '000480',
 'CS홀딩스': '000590',
 'DB': '012030',
 'DB증권': '016610',
 'DB손해보험': '005830',
 'DB하이텍': '000990',
 'iM금융지주': '139130',
 'DI동일': '001530',
 'DL': '000210',
 'DL우': '000215',
 'DL이앤씨': '375500',
 'DL이앤씨2우(전환)': '37550L',
 'DL이앤씨우': '37550K',
 'DN오토모티브': '007340',
 'DRB동일': '004840',
 'DSR': '155660',
 'DSR제강': '069730',
 'DS단석': '017860',
 'E1': '017940',
 'ESR켄달스퀘어리츠': '365550',
 'F&F': '383220',
 'F&F홀딩스': '007700',
 'GKL': '114090',
 'GS': '078930',
 'GS건설': '006360',
 'GS글로벌': '001250',
 'GS리테일': '007070',
 'GS우': '078935',
 'GS피앤엘': '499790',
 'HDC': '012630',
 'HDC랩스': '039570',
 'HDC현대EP': '089470',
 'HDC현대산업개발': '294870',
 'HD한국조선해양': '009540

In [55]:
newsMarketPipeline.add_ticker()

[파생] ticker 컬럼 생성
[파생] KOSPI 티커-이름 맵핑 생성
[파생] 티커-이름 맵핑 완료: 961개
[파생] ticker 컬럼 생성 완료


In [56]:
newsMarketPipeline.get_trading_days()

[파생] 거래일 리스트 생성


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2025-07: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-08: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-09: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-10: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2025-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-01: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-02: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-03: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-04: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2026-05: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-06: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-07: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-08: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2026-09: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-10: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
[파생] 거래일 리스트 생성 완료


DatetimeIndex(['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06',
               '2022-01-07', '2022-01-10', '2022-01-11', '2022-01-12',
               '2022-01-13', '2022-01-14',
               ...
               '2025-05-23', '2025-05-26', '2025-05-27', '2025-05-28',
               '2025-05-29', '2025-05-30', '2025-06-02', '2025-06-04',
               '2025-06-05', '2025-06-09'],
              dtype='datetime64[ns]', length=838, freq=None)

In [58]:
newsMarketPipeline.add_trading_dates()

[파생] 거래일 기준 컬럼 생성
[파생] 거래일 리스트 생성


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2025-07: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-08: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-09: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-10: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2025-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2025-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-01: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-02: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-03: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-04: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


...

2026-05: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-06: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-07: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-08: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-09: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)
/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: Deprec

2026-10: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-11: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
2026-12: 거래일 없음 또는 에러 ('RangeIndex' object has no attribute 'month')
[파생] 거래일 리스트 생성 완료


/var/folders/qp/j1r0p9s97s16j9gzg30pscb40000gn/T/ipykernel_79027/883097497.py:92: DeprecationWarning: Call to deprecated function (or staticmethod) get_business_days. (You should use get_previous_business_days() instead) -- Deprecated since version 1.1.
  days_this_month = stock.get_business_days(y, m)


[파생] 거래일 기준 컬럼 생성 완료


In [59]:
newsMarketPipeline.fetch_ohlcv_and_trading()

[수집] OHLCV/투자자별 순매수 데이터 수집
[수집] OHLCV/투자자별 순매수 데이터 수집 완료


In [60]:
newsMarketPipeline.add_ohlcv_and_trading()

[파생] OHLCV/투자자별 순매수 컬럼 생성
[파생] OHLCV/투자자별 순매수 컬럼 생성 완료


In [61]:
newsMarketPipeline.add_external_vars()

[병합] 외부 변수(환율, 국채, 기준금리) 병합
[수집] 환율 데이터
[수집] 환율 데이터 수집 완료
[수집] 국채10년 데이터
[수집] 국채10년 데이터 수집 완료
[수집] 기준금리 데이터
[수집] 기준금리 데이터 수집 완료
[병합] 외부 변수 병합 완료


In [63]:
newsMarketPipeline.get_df().head()

,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,stock_list,...,D_plus_14_date_volume,D_plus_14_date_foreign,D_plus_14_date_institution,D_plus_14_date_individual,D_day_date_open,D-day_change_open,D-day_change,fx,bond10y,base_rate
0,20240101_0049,2024-01-01 06:03:00,태영건설 파장 속 PF-ABCP '거래 부진'…차환 우려 고개,"PF 구조조정에 건설업 기피심리↑…""익숙한 악재라 파장 적을 것"" 관측도\n워크아웃...",연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2024/01/...,시공능력 순위 16위의 중견기업인 태영건설이 워크아웃을 신청하면서 부동산 PF 부실...,시공능력 순위 16위의 중견기업인 태영건설이 워크아웃을 신청하면서 부동산 PF 부실...,['태영건설'],...,632513.0,2.411097e+08,2.445000e+03,-2.413366e+08,4638.0,14.920224,13.163482,NaN,NaN,3.5
1,20240101_0001,2024-01-01 18:30:00,통합 셀트리온 12일 상장… 잭팟 터지나,합병 전 외국인·기관 동반 매수\n올해 영업익 1조5130억 전망\n최근 증시 상승...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/01/...,새해 출범하는 '통합 셀트리온'에 대한 기대감이 커지는 것으로 풀이된다. 외국인과 ...,외국인과 기관이 지난해 12월 삼성전자 주식을 1조4267억원어치 사들이며 전체 주...,['삼성전자'],...,19673375.0,1.372444e+11,-1.237386e+10,-1.303683e+11,78200.0,1.790281,1.401274,NaN,NaN,3.5
2,20240101_0032,2024-01-01 11:21:00,"외인·기관 홀린 '합병 셀트리온', 새해 잭팟 터트리나","[파이낸셜뉴스] 최근 증시 상승을 이끈 외국인, 기관 투자자들이 셀트리온그룹주를 순...",파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/01/...,새해 출범하는 '합병 셀트리온'에 대한 기대감이 커지는 것으로 풀이된다. 외국인과 ...,외국인과 기관이 지난해 12월 삼성전자 주식을 1조4267억원어치 사들이며 전체 주...,"['삼성전자', '셀트리온']",...,1153019.0,-2.332484e+10,3.074480e+10,-7.904098e+09,187209.0,13.760022,14.888601,NaN,NaN,3.5
3,20240101_0017,2024-01-01 17:45:00,"김·김치·라면 '수출 대박'…""올핸 떡볶이·핫도그가 대열 합류""",■'차세대 한류' 노리는 식품업계\n라면·김 작년 수출액 1조원 돌파\n역대급 실적...,서울경제,https://n.news.naver.com/mnews/article/011/000...,https://imgnews.pstatic.net/image/011/2024/01/...,■'차세대 한류' 노리는 식품업계 라면·김 작년 수출액 1조원 돌파 역대급 실적에 ...,"CJ 제일001040) 제일제당은 유럽, 호주 등에서 영토를 넓힘과 동시에 오카도,...",['농심'],...,15579.0,1.764586e+09,-1.128099e+09,-6.181225e+08,407000.0,1.351351,1.351351,NaN,NaN,3.5
4,20240101_0011,2024-01-01 18:15:00,"""빅딜 절호의 기회"" 새해부터 분주한 K제약바이오",'JP모건 헬스케어 컨퍼런스'\n삼바·셀트리온 등 대거 참여\n기술력 알리고 협력기...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://ssl.pstatic.net/static.news/image/news...,'JP모건 헬스케어 컨퍼런스' 삼바·셀트리온 등 대거 참여 기술력 알리고 협력기회 ...,오는 8일부터 미국 샌프란시스코에서 열리는 'JP모건 헬스케어 컨퍼런스'에 참여하는...,"['한미약품', '유한양행']",...,253583.0,3.324466e+09,-4.505191e+09,1.293182e+09,68800.0,-1.453488,-1.453488,NaN,NaN,3.5
